# Explainer for Iris model with Poetry-defined Environment

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * poetry
 * rclone
 * curl

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

We will assume that ambassador (or Istio) ingress is port-forwarded to `localhost:8003`

## Setup MinIO

Use the provided [notebook](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html) to install Minio in your cluster.
Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html).

We will assume that MinIO service is port-forwarded to `localhost:8090`

In [1]:
%%writefile rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://localhost:8090

Overwriting rclone.conf


In [2]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-rclone-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Overwriting secret.yaml


In [3]:
!kubectl apply -f secret.yaml

secret/seldon-rclone-secret configured


## Poetry

We will use `poetry.lock` to fully define the explainer environment. Install poetry following official [documentation](https://python-poetry.org/docs/#installation). Usually this goes down to
```bash
curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python3 -
```

# Deploy Iris Model

In [4]:
%%writefile iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/v1.15.0-dev/sklearn/iris

Overwriting iris.yaml


In [5]:
!kubectl apply -f iris.yaml

seldondeployment.machinelearning.seldon.io/iris configured


In [6]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=iris -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "iris-default-0-classifier" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "iris-default-0-classifier" rollout to finish: 1 old replicas are pending termination...
deployment "iris-default-0-classifier" successfully rolled out


In [7]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/iris/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364059,
        0.04505474761562512,
        5.7927447968953825e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.15.0-dev"
    }
  }
}


# Train Explainer

## Prepare Training Environment

We are going to use `pyproject.toml` and `poetry.lock` files from [Alibi Explain Server](https://github.com/SeldonIO/seldon-core/tree/master/components/alibi-explain-server). This will allow us to create environment that will match the runtime one.

In [8]:
%%bash
cp ../../../components/alibi-explain-server/pyproject.toml .
cp ../../../components/alibi-explain-server/poetry.lock .

conda create --yes --prefix ./venv python=3.7.10

Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/rskolasinski/work/seldon-core/examples/explainers/iris-explainer-poetry/venv

  added / updated specs:
    - conda-ecosystem-user-package-isolation
    - python=3.7.10


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu
  ca-certificates    conda-forge/linux-64::ca-certificates-2022.5.18.1-ha878542_0
  conda-ecosystem-u~ conda-forge/linux-64::conda-ecosystem-user-package-isolation-1.0-ha770c72_1
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.36.1-hea4e1c9_2
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5
  libgcc-ng          conda-forge/linux-64::libgcc-ng-12.1.0-h8d9b700_16
  libgomp            conda-forge/l

In [9]:
%%bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate ./venv
poetry install

Installing dependencies from lock file

Package operations: 138 installs, 0 updates, 0 removals

  • Installing certifi (2021.10.8)
  • Installing charset-normalizer (2.0.11)
  • Installing idna (3.3)
  • Installing pyasn1 (0.4.8)
  • Installing pycparser (2.21)
  • Installing typing-extensions (4.0.1)
  • Installing urllib3 (1.26.8)
  • Installing zipp (3.7.0)
  • Installing cachetools (5.0.0)
  • Installing cffi (1.15.0)
  • Installing cymem (2.0.6)
  • Installing importlib-metadata (4.10.1)
  • Installing numpy (1.19.5)
  • Installing oauthlib (3.2.0)
  • Installing pyparsing (3.0.7)
  • Installing requests (2.27.1)
  • Installing rsa (4.7.2)
  • Installing six (1.16.0)
  • Installing pyasn1-modules (0.2.8)
  • Installing murmurhash (1.0.6)
  • Installing blis (0.7.5)
  • Installing catalogue (1.0.0)
  • Installing click (8.0.3)
  • Installing cryptography (36.0.1)
  • Installing filelock (3.4.2)
  • Installing google-auth (2.6.0)
  • Installing joblib (1.1.0)
  • Installing packagi

## Prepare Training Script

In [10]:
%%writefile train.py
import numpy as np
from sklearn.datasets import load_iris
from alibi.explainers import AnchorTabular

import requests


dataset = load_iris()
feature_names = dataset.feature_names
iris_data = dataset.data

model_url = "http://localhost:8003/seldon/seldon/iris/api/v1.0/predictions"


def predict_fn(X):
    data = {"data": {"ndarray": X.tolist()}}
    r = requests.post(model_url, json={"data": {"ndarray": [[1, 2, 3, 4]]}})
    return np.array(r.json()["data"]["ndarray"])


explainer = AnchorTabular(predict_fn, feature_names)
explainer.fit(iris_data, disc_perc=(25, 50, 75))

explainer.save("./explainer/")

Overwriting train.py


In [11]:
%%bash
unset MPLBACKEND # required as we call the script from Jupyter Lab in this demo
./venv/bin/python3 train.py

IPython could not be loaded!


In [12]:
!tree explainer/

explainer/
├── explainer.dill
└── meta.dill

0 directories, 2 files


# Save and deploy Explainer

In [13]:
!rclone --config="rclone.conf" copy explainer/ s3:explainers/iris/

In [14]:
%%writefile iris-with-explainer.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/v1.11.0-dev/sklearn/iris
    explainer:
      type: AnchorTabular
      modelUri: s3:explainers/iris/
      envSecretRefName: seldon-rclone-secret
      replicas: 1            

Overwriting iris-with-explainer.yaml


In [15]:
!kubectl apply -f iris-with-explainer.yaml

seldondeployment.machinelearning.seldon.io/iris configured


In [16]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=iris -o jsonpath='{.items[0].metadata.name}')
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=iris -o jsonpath='{.items[1].metadata.name}')

Waiting for deployment "iris-default-0-classifier" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "iris-default-0-classifier" rollout to finish: 1 old replicas are pending termination...
deployment "iris-default-0-classifier" successfully rolled out
deployment "iris-default-explainer" successfully rolled out


# Test Deployed explainer

In [17]:
import numpy as np
import requests

In [18]:
model_url = (
    "http://localhost:8003/seldon/seldon/iris-explainer/default/api/v1.0/explain"
)


def explain_fn(X):
    data = {"data": {"ndarray": X.tolist()}}
    r = requests.post(model_url, json={"data": {"ndarray": [[1, 2, 3, 4]]}})
    return r.json()

In [19]:
explanation = explain_fn(np.array([[5.964, 4.006, 2.081, 1.031]]))

In [20]:
print("Anchor: %s" % (" AND ".join(explanation["data"]["anchor"])))
print("Precision: %.2f" % explanation["data"]["precision"])
print("Coverage: %.2f" % explanation["data"]["coverage"])

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.97
Coverage: 0.08
